# Proposal

The current readout seems to have a sharp roll-off as the frequency approaches 4 GHz. Let's keep the highest resonance frequency below 3.8 GHz, neglecting kinetic inductance. This should allow us to test all-niobium resonators.

Split this band into four sub-bands. Since the expected loading is higher at 220 GHz and we probably need more active volume to keep $Q_i$ as high as possible, plan for the 220 bands to have longer active regions. To maintain approximately the same active-to-inactive ratio, place these bands at lower resonance frequencies.

Set the resonance spacing to 8 MHz, which is 20 linewidths for a $Q = 10^4$ resonator at 4 GHz. Each 23-resonator band will then occupy about 200 MHz. (We should really keep the fractional spacing constant, but I think we have enough bandwidth that this doesn't matter.) With 50 MHz gaps between the bands, we need about 250 MHz per 23-resonance band.

This proposal would let us read out one sub-band in heterodyne mode with both negative and positive frequencies, or slightly less than one sub-band using only positive frequencies.

Since we don't know the kinetic inductance fractions a priori, and since they will vary somewhat between the bands and designs, start by writing the length as a function of desired resonance frequency.

In [1]:
from scipy.constants import c

The resonance frequency of a quarter-wavelength line resonator is approximately
$$
f_r = (1 - \alpha)^{1/2} \frac{c}{4 \ell \epsilon_e^{1/2}},
$$
where $\alpha$ is the kinetic inductance fraction, $c$ is the speed of light, $\ell$ is the length of the resonator, and $\epsilon_e$ is the effective dielectric constant. For a planar structure on a substrate with dielectric constant $\epsilon_s$, the effective dielectric constant is the average of the substrate and vacuum values:
$$
\epsilon_e = (1 + \epsilon_s) / 2.
$$

In [2]:
def resonance(length, alpha=0, epsilon_substrate=11.9):
    epsilon_effective = (1 + epsilon_substrate) / 2
    v = c / epsilon_effective**(1/2)
    wavelength = 4 * length
    f_r0 = v / wavelength
    f_r = (1 - alpha)**(1/2) * f_r0
    return f_r

def length(f_r, alpha=0, epsilon_substrate=11.9):
    epsilon_vacuum = 1
    epsilon_effective = (epsilon_vacuum + epsilon_substrate) / 2
    return (1 - alpha)**(1/2) * c / (4 * f_r * epsilon_effective**(1/2))

In [3]:
Q = 1e4
f_r = 4e9
df = f_r / Q
linewidths = 20
print("The nominal resonance spacing is {:.1f} MHz".format(1e-6 * df * linewidths))

The nominal resonance spacing is 8.0 MHz


In [4]:
resonators_per_band = 23
F = df * linewidths
B = F * resonators_per_band 
print("With {} resonators per band, each band occupies {:.0f} MHz".format(resonators_per_band, 1e-6 * B))

With 23 resonators per band, each band occupies 184 MHz


In [37]:
f_max = 3200e6  # The maximum resonance frequency
between = 50e6  # The frequency spacing between bands

In [38]:
high150 = np.linspace(f_max - B, f_max, resonators_per_band)
low150 = np.linspace(high150[0] - between - B, high150[0] - between, resonators_per_band)
high220 = np.linspace(low150[0] - between - B, low150[0] - between, resonators_per_band)
low220 = np.linspace(high220[0] - between - B, high220[0] - between, resonators_per_band)

In [39]:
print("The low-frequency 220 GHz band covers {:.0f} to {:.0f} MHz.".format(1e-6 * low220[0], 1e-6 * low220[-1]))
print("The high-frequency 220 GHz band covers {:.0f} to {:.0f} MHz.".format(1e-6 * high220[0], 1e-6 * high220[-1]))
print("The low-frequency 150 GHz band covers {:.0f} to {:.0f} MHz.".format(1e-6 * low150[0], 1e-6 * low150[-1]))
print("The high-frequency 150 GHz band covers {:.0f} to {:.0f} MHz.".format(1e-6 * high150[0], 1e-6 * high150[-1]))

The low-frequency 220 GHz band covers 2314 to 2498 MHz.
The high-frequency 220 GHz band covers 2548 to 2732 MHz.
The low-frequency 150 GHz band covers 2782 to 2966 MHz.
The high-frequency 150 GHz band covers 3016 to 3200 MHz.


In [44]:
1e3 * length(low220, alpha=0.2)

array([ 11.40676497,  11.36568525,  11.32490036,  11.28440713,
        11.24420244,  11.20428322,  11.16464645,  11.12528912,
        11.0862083 ,  11.04740109,  11.00886462,  10.97059607,
        10.93259265,  10.89485161,  10.85737026,  10.82014592,
        10.78317594,  10.74645775,  10.70998877,  10.67376647,
        10.63778836,  10.60205198,  10.5665549 ])

In [45]:
1e3 * length(high220, alpha=0.2)

array([ 10.35920492,  10.32531278,  10.29164169,  10.2581895 ,
        10.22495406,  10.19193329,  10.1591251 ,  10.12652747,
        10.09413835,  10.06195576,  10.02997774,   9.99820232,
         9.96662761,   9.93525169,   9.90407271,   9.8730888 ,
         9.84229815,   9.81169896,   9.78128943,   9.75106782,
         9.72103239,   9.69118142,   9.66151323])

In [46]:
1e3 * length(low150, alpha=0.2)

array([ 9.48786993,  9.45943166,  9.43116337,  9.40306352,  9.37513063,
        9.34736319,  9.31975976,  9.29231887,  9.26503911,  9.23791904,
        9.21095728,  9.18415245,  9.15750317,  9.1310081 ,  9.1046659 ,
        9.07847525,  9.05243485,  9.02654342,  9.00079966,  8.97520233,
        8.94975018,  8.92444198,  8.89927651])

In [47]:
1e3 * length(high150, alpha=0.2)

array([ 8.75174209,  8.72753984,  8.70347109,  8.67953472,  8.65572965,
        8.63205481,  8.60850912,  8.58509153,  8.561801  ,  8.5386365 ,
        8.51559701,  8.49268151,  8.46988902,  8.44721854,  8.42466909,
        8.40223971,  8.37992945,  8.35773735,  8.33566248,  8.31370391,
        8.29186073,  8.27013204,  8.24851692])

In [36]:
[1e6 * length(a).ptp() for a in [low220, high220, low150, high150]]

[742.41268531070182,
 628.82455966035627,
 539.44834185332309,
 467.86020961634216]